Import libraries

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

Load csv-file with txn and transform format 3 fields

In [3]:
myfile = 'transactions_2.csv'
transactions = pd.read_csv(myfile, sep=';', header=0)
transactions.loc[:,'blockTimestamp'] = pd.to_datetime(transactions['blockTimestamp'])
transactions.loc[:,'gas'] = transactions['gas'].map(lambda x: int(x,16))
transactions.loc[:,'gasPrice'] = transactions['gasPrice'].map(lambda x: int(x,16))

Aggregate of data for addresses "from"

In [4]:
address_from_blockTimestamp = transactions.groupby('from')['blockTimestamp'].agg({'count':np.count_nonzero, 
                                                                                  'delta_timestamp': lambda x: 
                                                                                       (np.max(x) - np.min(x)).total_seconds(),
                                                                                 'min_timestamp':np.min,
                                                                                 'max_timestamp':np.max}).reset_index()

address_from_value = transactions.groupby('from')['value'].agg({'count_value_non_zero':np.count_nonzero, 
                                                                'mean_value':np.mean, 
                                                                'max_value':np.max, 
                                                                'sum_value':np.sum}).reset_index()

address_from_gas = transactions.groupby('from')['gas'].agg({'count_non_zero_gas':np.count_nonzero, 
                                                            'min_gas':np.min, 'mean_gas':np.mean, 
                                                            'max_gas':np.max, 
                                                            'sum_gas':np.sum}).reset_index()

address_from_gasPrice = transactions.groupby('from')['gasPrice'].agg({'count_non_zero_gasPrise':np.count_nonzero, 
                                                                      'min_gasPrice':np.min, 
                                                                      'mean_gasPrice':np.mean, 
                                                                      'max_gasPrice':np.max, 
                                                                      'sum_gasPrice':np.sum}).reset_index()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:15: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  from ipykernel import kernelapp as app
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version


Aggregate of data for addresses "to"

In [9]:
address_to_blockTimestamp = transactions.groupby('to')['blockTimestamp'].agg({'count':np.count_nonzero, 
                                                                              'delta_timestamp': lambda x: 
                                                                                       (np.max(x) - np.min(x)).total_seconds(),
                                                                              'min_timestamp':np.min,
                                                                              'max_timestamp':np.max}).reset_index()

address_to_value = transactions.groupby('to')['value'].agg({'count_value_non_zero':np.count_nonzero, 
                                                            'mean_value':np.mean, 
                                                            'max_value':np.max, 
                                                            'sum_value':np.sum}).reset_index()

address_to_gas = transactions.groupby('to')['gas'].agg({'count_non_zero_gas':np.count_nonzero, 
                                                        'min_gas':np.min, 
                                                        'mean_gas':np.mean, 
                                                        'max_gas':np.max, 
                                                        'sum_gas':np.sum}).reset_index()

address_to_gasPrice = transactions.groupby('to')['gasPrice'].agg({'count_non_zero_gasPrise':np.count_nonzero, 
                                                                  'min_gasPrice':np.min, 
                                                                  'mean_gasPrice':np.mean, 
                                                                  'max_gasPrice':np.max, 
                                                                  'sum_gasPrice':np.sum}).reset_index()

address_is_contract = transactions.groupby('to')['input'].agg({'is_contract': lambda x: 
                                                               False if len(np.unique(x))>1 
                                                                         or np.unique(x)=='0x' else True}).reset_index()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:22: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version


Merging aggregated DataFrames to one.  <br>
You may uncoment rows for <b>outer-join</b>

In [10]:
address_from = pd.merge(pd.merge(pd.merge(address_from_blockTimestamp, 
                                              address_from_value,
                                              on='from'),
                                     address_from_gas,
                                     on='from'),
                             address_from_gasPrice,
                             on='from')

address_to = pd.merge(pd.merge(pd.merge(pd.merge(address_to_blockTimestamp, 
                                                      address_to_value,
                                                      on='to'),
                                             address_is_contract,
                                             on='to'),
                                     address_to_gas,
                                     on='to'),
                             address_to_gasPrice,
                             on='to')

addresses_analysis = pd.merge(address_from, 
                                 address_to,
#                                  how='outer', 
                                 left_on='from', 
                                 right_on='to', 
                                 suffixes=('_from', '_to'))

# addresses_to_analysis.fillna(0, inplace=True)

Calculating of address field

In [12]:
def address_function(x):
    return x['from'] if  x['from']==x['from'] else x['to']

addresses_analysis.loc[:,'address'] = addresses_analysis.apply(address_function, axis=1)
addresses_analysis.drop(['from','to'], inplace=True, axis=1)

Save results

In [13]:
addresses_analysis.to_csv('addresses_analysis.csv')
# address_to.to_csv('addresses_to.csv')
# address_from.to_csv('addresses_from.csv')

In [5]:
address_to_blockTimestamp

NameError: name 'address_to_blockTimestamp' is not defined